In [ ]:
---
title: "Trabajo final Master - Ntic/UCM"
author: "Leonardo Valverde Badilla"
date: "14 de febrero del 2021"
output: html_document
---

#### **Índice**

1. [Introducción](#id1)
2. [Descripción del problema](#id2)
3. [Análisis Exploratorio](#id3)
4. [Dummies](#id4)
5. [Estandarización](#id5)
6. [Selección de variables](#id6)
7. [Modelización](#id7)
7. [Ensamblado](#id8)

<div id='id1' />

#### **Introducción**
Ntic Master y la Universidad Complutense de Madrid tienen el master en "Big data and business analytics" donde el trabajo final del master es sobre "Credit Card Customers", el cual el gerente del banco está molesto porque cada vez más clientes abandonan sus servicios de tarjetas de crédito. Él desea predecir  quién va a abandonar la tarjeta de crédito para que de manera proactiva le puedan ofrecer un mejor servicio a los clientes, así puedan cambiar de opinión y no abandonen el banco.

La fuente de los datos se obtuvo de Kaggle: https://www.kaggle.com/sakshigoyal7/credit-card-customers

<div id='id2' />

#### **Descripción del problema**

Como se mencionó en el encabezado, el banco está reportando un mayor abandono de los clientes de la tarjeta de crédito y el gerente desea predecir el abandono de cada cliente para mejorarle los servicios y no abandone el banco.

En conclusión, se deben conocer las características de los clientes que abandonan los servicios de la tarjeta de crédito para que no aumente este KPI dentro de la organización, esto puede afectar otros indicadores como rentabilidad, mala imagen por servicio, entre otros.

```{r, warning=FALSE, echo=FALSE,message=FALSE}
library(doParallel)
registerDoParallel(makeCluster(6))
```

<div id='id3' />

#### **Análisis Exploratorio**

Se carga la base y se presenta una pequeña parte de las 10.217 observaciones y 21 variables:

```{r, echo=FALSE, message=FALSE, warning=FALSE}
setwd("C:/Users/lvalverde/Documents/UCM/TFM")
base <- read.csv("Credit_Card_Customers.csv")

dim(base)

library(kableExtra)
kable(base[1:5,2:9]) %>%
  kable_styling("striped", full_width = F)
```

El formato de las 21 variables se resumen de la siguiente manera:

* **ID**: ClientNum (El número de cliente)
* **Target**: Attrition_Flag (Variable Objetivo)
* **Character**: 
  + Gender (Género: Femenino y Masculino)
  + Education level (Nivel de educación)
  + Marital Status (Estado civil)
  + Income Category (Categoría de ingreso)
  + Card Category (Categoría de la tarjeta de crédito)
* **Numérico**: 
  + Age (Edad)
  + Dependent Count (Cantidad de dependientes)
  + Month on book (Periodo de relación con el banco)
  + Total Relationship Count (Cantidad de productos que posee el cliente)
  + Months Inactive 12 mon (Número de meses inactivos en los últimos 12 meses)
  + Contacts Count 12 mon (Cantidad de contactos en los U12M)
  + Credit Limit (Límite de crédito)
  + Total Revolving Bal (Saldo giratorio de la tarjeta de crédito)
  + Avg Open To Buy (Línea de crédito para compras (Prom. U12M))
  + Total Amt Chng Q4 Q1 (Cambio en el monto de transacción)
  + Total Trans Amt (Monto total de transacciones U12M)
  + Total Trans Ct (Cantidad total de transacciones U12M)
  + Total Ct Chng Q4 Q1 (Cambio en lacantidad de transacciones)
  + Avg Utilization Ratio (Índice de utilización promedio de la tarjeta)

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(ggplot2)
library(dfexplore)
dfplot(base[,c(3:21)], title = "Credit Card Customers")
```

```{r, echo=FALSE, message=FALSE, warning=FALSE}
base$Attrition_Flag <- as.factor(base$Attrition_Flag)
base$Gender <- as.factor(base$Gender)
base$Education_Level <- as.factor(base$Education_Level)
base$Marital_Status <- as.factor(base$Marital_Status)
base$Income_Category <- as.factor(base$Income_Category)
base$Card_Category <- as.factor(base$Card_Category)
```

La función "Summary" nos permite  ver el resumen estadístico de las variables donde podemos analizar lo siguiente: 

- Las variables "Education level", "Martital Status" y "Income_Category" tienen datos desconocidos, es decir, podríamos sustituirlos por missings para validar el porcentaje de nulos y saber qué validación podemos aplicar en estas tres variables.

- Se reportan valores máximos con gran diferencia entre la media en las variables numéricas de "Avg Open To Buy", "Credit Limit", "Total Amt Chng Q4_Q1", "Total Amt Chng Q4_Q1" y "Total Trans Amt", por lo que se pueden realizar boxplots para identificar outliers.
  
- Se debe sustituir los "Unknown" por missings para después hacer las validaciones correspondientes de los nulos ya que no son una categoría dentro de cada variable.

```{r, echo=FALSE, warning=FALSE, message=FALSE}
summary(base)
```
Con respecto a los valores máximos anteriores, se procede a validar las 4 variables numéricas para saber si se reportan valores atípicos, y se evidencia outliers principalmente en Total Ct Ching Q4-Q1 donde a partir de 3 ya que esto podría introducir un sesgo.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
par(bty='n', mfrow=c(2,2))
boxplot(base$Total_Ct_Chng_Q4_Q1, main = "Boxplot Total Ct Ching Q4-Q1")
boxplot(base$Avg_Open_To_Buy, main = "Boxplot Avg Open to buy")
boxplot(base$Total_Trans_Amt, main = "Boxplot Total Trans Amt")
boxplot(base$Credit_Limit, main = "Boxplot Credit Limit")
```

Se procede a sustituir los outliers de "Total CT Chng Q4-Q1" y los valores "Unkhown" para evitar un sesgo de los datos, y se recodifica la variable objetivo para ser "Yes" o "No".

```{r, echo=FALSE, message=FALSE, warning=FALSE}
base$Total_Ct_Chng_Q4_Q1 <- replace(base$Total_Ct_Chng_Q4_Q1, which((base$Total_Ct_Chng_Q4_Q1 >= 3)), NA)
base$Education_Level <- replace(base$Education_Level, which((base$Education_Level == "Unknown")), NA)
base$Marital_Status <- replace(base$Marital_Status, which((base$Marital_Status == "Unknown")), NA)
base$Income_Category <- replace(base$Income_Category, which((base$Income_Category == "Unknown")), NA)
base$Attrition_Flag <- ifelse(base$Attrition_Flag=="Attrited Customer","Yes","No")
base$Attrition_Flag <- as.factor(base$Attrition_Flag)
summary(base[,c(2,6:8,20)])
```

Se hace un análisis de algunas de las variables numéricas:

- El promedio de uso de la tarjeta de crédito se destaca porque la mayoría de los clientes están por debajo del 10%.
- El mayor rango de edad de los clientes está entre 40 a 50 años.
- La cantidad de meses inactivos en el último año es entre 1 a 3 meses.
- Existen más clientes con un límite de crédito inferior a los 5 mil dólares.


```{r, echo=FALSE, message=FALSE, warning=FALSE}
par(mfrow=c(2,3))

hist(base$Credit_Limit, col = heat.colors(20),
     xlab = "Límite crédito", ylab = "Cantidad Clientes", main = "Clientes por límite de crédito")

hist(base$Customer_Age, col = "green",
     xlab = "Edad", ylab = "Cantidad Clientes", main = "Edad de los clientes")

hist(base$Total_Trans_Ct, col = colorRampPalette(c('blue', 'red'))(15),
     xlab = "Cantidad transacciones", ylab = "Cantidad Clientes", main = "Total de transacciones del último año")

hist(base$Months_Inactive_12_mon, col = "darkblue",
     xlab = "Meses inactivos", ylab = "Cantidad Clientes", main = "Meses inactivos en el último año")

hist(base$Total_Relationship_Count, col = 'darkgreen', border = "white",
     xlab = "Cantidad productos", ylab = "Cantidad Clientes", main = "Cantidad productos del cliente")

hist(base$Avg_Utilization_Ratio, col = terrain.colors(20),
     xlab = "Utilización tarjeta", ylab = "Cantidad Clientes", main = "Promedio del uso de la tarjeta")
```

Se valida la cantidad de missings donde se evidencia que de las 21 variables, 4 poseen missings, donde la variable "Education Level" es la que posee más missings donde el 15% de las observaciones son nulos.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(naniar)
gg_miss_var(base)
```

```{r, echo=FALSE, message=FALSE, warning=FALSE}
row.names(base) <- base$CLIENTNUM
base2 <- base[,-1]
```

Se procede a imputar las variables con missings ya que si se eliminan de la base de datos, se exluye el 30% de las observaciones, así que la recomendación es imputarlos con la moda de cada atributo. Por lo que queda de la siguiente manera:

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(psych)
base2$prop_missings<-apply(is.na(base2),1,mean)
summary(base2$prop_missings)
```

```{r, echo=FALSE, message=FALSE, warning=FALSE}
listclass <- c("Gender", "Marital_Status", "Education_Level", "Income_Category", "Card_Category")
listnumeric <- c("Customer_Age", "Dependent_count", "Months_on_book", "Total_Relationship_Count", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Credit_Limit", "Total_Revolving_Bal", "Avg_Open_To_Buy", "Total_Amt_Chng_Q4_Q1", "Total_Trans_Amt", "Total_Trans_Ct", "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio")
vardep <- "Attrition_Flag"
```


```{r, echo=FALSE, warning=FALSE, message=FALSE}
#Missing de variables numericas
tablamis1<-as.data.frame(sapply(base2[,listnumeric],function(x) sum(is.na(x))))
names(tablamis1)[1]<-"nmiss"

#Missing de variables categoricas
tablamis2<-as.data.frame(sapply(base2[,listclass],function(x) sum(is.na(x))))
names(tablamis2)[1]<-"nmiss"

#Missing total
tablatotal<-as.data.frame(rbind(tablamis2,tablamis1))
tablatotal2 <- subset(tablatotal, tablatotal>0)
```

```{r, echo=FALSE, message=FALSE, warning=FALSE}
factores<-names(Filter(is.factor, base2))
tablamis<-as.data.frame(sapply(base2[,factores],function(x) sum(is.na(x))))
names(tablamis)[1]<-"nmiss"

tablatotal<-as.data.frame(tablamis)
tablatotal2 <- subset(tablatotal, tablatotal$nmiss>0)

tablatotal2$nombrevar<-row.names(tablatotal2)
lista<-tablatotal2[which(tablatotal2$nmiss>0),]
listamisimp<-dput(lista$nombrevar)

lista2<-intersect(listamisimp, factores)

Mode <- function(x){
  a = table(x)
  return(names(a[which.max(a)]))
}

moda_categorica <- as.data.frame(sapply(base2[,lista2],function(x) ifelse(is.na(x),Mode(x),x)))
```

```{r, echo=FALSE, message=FALSE, warning=FALSE}
base2$Education_Level[is.na(base2$Education_Level)] <- "Graduate"
base2$Marital_Status[is.na(base2$Marital_Status)] <- "Married"
base2$Income_Category[is.na(base2$Income_Category)] <- "Less than $40K"
summary(base2[,c(5:7)])
```


| Atributo | Observaciones | Moda |
|------- |------------- |--------- |
| Education_Level | 1519 | 	Graduate |
| Marital_Status | 749 | Married |
| Income_Category | 1112 | Less than $40K |


Antes de realizar los dummies, necesitamos eliminar las observaciones con Missings donde al realizar este ejercicio únicamente estaríamos elimnando los 6 observaciones.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
base3<-na.omit(base2,(!is.na(base2)))
dim(base3)
texto <- paste("Total de observaciones eliminadas:", (nrow(base2)-nrow(base3)),sep = " ")
texto
base4 <- base3[,-21]
```

<div id='id4' />

#### **Dummies**

EL siguiente paso es crear dummies para poder realizar el estudio de variables y definir las más importantes, donde este paso nos generó 15 atributos nuevos ya que se hizo únicamente para variables categóricas. 

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(dummies)
base5<-dummy.data.frame(base4, listclass, sep = ".")
dim(base5)

texto2 <- paste("El modelo generó", (ncol(base5)-ncol(base4)), "nuevos atributos",sep = " ")
texto2
```

Después de realizar los dummies, se valida la participación de los atributos generados, esto para definir si dejamos todos los dummies o no, ejemplo: Género se dividió en 2 (Género.F y Género.M) por lo que si dejamos Género.F entonces si trae un cero se puede definir como masculino (Ya que no existe otro género dentro de la base).

**Nota**: Únicamente se va a realizar una pequeña comprobación de lo anterior, ya que es preferible usar técnicas de Machine Learning que nos indica la importancia de las variables.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
par(mfrow=c(2,2))
barplot(table(base4$Education_Level), col = terrain.colors(10))
barplot(table(base4$Marital_Status), col = heat.colors(5))
barplot(table(base4$Income_Category), col = terrain.colors(15))
barplot(table(base4$Card_Category), col = "darkblue")
```

**Education Level**: El 45.9% de las observaciones son "Graduate", sin embargo, podríamos excluir únicamente los atributos "Doctorate" y "Post-Graduate" ya que solo el 5% de los clientes poseen estos grados académicos.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(htmlTable)

educacion_college <- as.data.frame(round((prop.table(table(base5$Education_Level.College))*100),1))
names(educacion_college) <- c("Categoria", "College")
educacion_doctorate <- as.data.frame(round((prop.table(table(base5$Education_Level.Doctorate))*100),1))
names(educacion_doctorate) <- c("Categoria", "Doctorate")
educacion_graduate <- as.data.frame(round((prop.table(table(base5$Education_Level.Graduate))*100),1))
names(educacion_graduate) <- c("Categoria", "Graduate")
educacion_hs <- as.data.frame(round((prop.table(table(base5$`Education_Level.High School`))*100),1))
names(educacion_hs) <- c("Categoria", "High_School")
educacion_post <- as.data.frame(round((prop.table(table(base5$`Education_Level.Post-Graduate`))*100),1))
names(educacion_post) <- c("Categoria", "Post-Graduate")
educacion_uneducated <- as.data.frame(round((prop.table(table(base5$Education_Level.Uneducated))*100),1))
names(educacion_uneducated) <- c("Categoria", "Uneducated")

freq_education <- merge(educacion_college, educacion_doctorate)
freq_education2 <- merge(freq_education, educacion_graduate)
freq_education3 <- merge(freq_education2, educacion_hs)
freq_education4 <- merge(freq_education3, educacion_post)
freq_education5 <- merge(freq_education4, educacion_uneducated)

htmlTable(head(freq_education5),
          caption = "Education Level",
          tfoot = "0 corresponde a 'No' y 1 a 'Si'")
```

**Marital Status**: El 53.7% de los clientes están casados, por lo que podemos dejar únicamente este estado ya que "Divorciado" tiene poca participación, por lo que si no está casado entonces podría estar soltero o divorciado.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
casado <- as.data.frame(round((prop.table(table(base5$Marital_Status.Married))*100),1))
names(casado) <- c("Categoria", "Casado")
soltero <- as.data.frame(round((prop.table(table(base5$Marital_Status.Single))*100),1))
names(soltero) <- c("Categoria", "Soltero")
divorciado <- as.data.frame(round((prop.table(table(base5$Marital_Status.Divorced))*100),1))
names(divorciado) <- c("Categoria", "Divorciado")

freq_estadocivil <- merge(casado, divorciado)
freq_estadocivil2 <- merge(freq_estadocivil, soltero)

htmlTable(head(freq_estadocivil2),
          caption = "Marial Status",
          tfoot = "0 corresponde a 'No' y 1 a 'Si'")
```

**Income Category**: El 46.1% reporta un ingreso inferior a los 40 mil anuales, sin embargo, hay poca participación en las demás categorías con mayor a 120 mil ya que es el 7.2%.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
inferior40 <- as.data.frame(round((prop.table(table(base5$`Income_Category.Less than $40K`))*100),1))
names(inferior40) <- c("Categoria", "Less than 40K")
income_40a60 <- as.data.frame(round((prop.table(table(base5$`Income_Category.$40K - $60K`))*100),1))
names(income_40a60) <- c("Categoria", "$40K - $60K")
income_60a80 <- as.data.frame(round((prop.table(table(base5$`Income_Category.$60K - $80K`))*100),1))
names(income_60a80) <- c("Categoria", "$60K - $80K")
income_80a120 <- as.data.frame(round((prop.table(table(base5$`Income_Category.$80K - $120K`))*100),1))
names(income_80a120) <- c("Categoria", "$80K - $120K")
income_120 <- as.data.frame(round((prop.table(table(base5$`Income_Category.$120K +`))*100),1))
names(income_120) <- c("Categoria", "$120K +")

freq_income <- merge(inferior40, income_40a60)
freq_income2 <- merge(freq_income, income_60a80)
freq_income3 <- merge(freq_income2, income_80a120)
freq_income4 <- merge(freq_income3, income_120)

htmlTable(head(freq_income4),
          caption = "Income Category",
          tfoot = "0 corresponde a 'No' y 1 a 'Si'")
```

**Card Category**: El 93.2% de los clientes poseen tareta de crédito Blue, por lo que es preferible dejar únicamente esta categoría.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
blue <- as.data.frame(round((prop.table(table(base5$Card_Category.Blue))*100),1))
names(blue) <- c("Categoría", "Blue")
gold <- as.data.frame(round((prop.table(table(base5$Card_Category.Gold))*100),1))
names(gold) <- c("Categoría", "Gold")
platinum <- as.data.frame(round((prop.table(table(base5$Card_Category.Platinum))*100),1))
names(platinum) <- c("Categoría", "Platinum")
silver <- as.data.frame(round((prop.table(table(base5$Card_Category.Silver))*100),1))
names(silver) <- c("Categoría", "Silver")

freq_card <- merge(blue, gold)
freq_card2 <- merge(freq_card, platinum)
freq_card3 <- merge(freq_card2, silver)

htmlTable(head(freq_card3),
          caption = "Card Category",
          tfoot = "0 corresponde a 'No' y 1 a 'Sí'")
```


```{r, echo=FALSE, message=FALSE, warning=FALSE}
listclass <- c("Gender.F", "Education_Level.College", "Education_Level.Doctorate","Education_Level.Graduate","Education_Level.High School","Education_Level.Post-Graduate", "Education_Level.Uneducated","Marital_Status.Divorced","Marital_Status.Married","Marital_Status.Single","Income_Category.$120K +","Income_Category.$40K - $60K","Income_Category.$60K - $80K","Income_Category.$80K - $120K","Income_Category.Less than $40K","Card_Category.Blue","Card_Category.Gold","Card_Category.Platinum","Card_Category.Silver")
listnumeric <- c("Customer_Age", "Dependent_count", "Months_on_book", "Total_Relationship_Count", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Credit_Limit", "Total_Revolving_Bal", "Avg_Open_To_Buy", "Total_Amt_Chng_Q4_Q1", "Total_Trans_Amt", "Total_Trans_Ct", "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio")
vardep <- "Attrition_Flag"
```


<div id='id5' />

#### **Estandarización**

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(MASS)
# Calculo medias y dtipica de datos y estandarizo (solo las continuas)
means <-apply(base5[,listnumeric],2,mean)
sds<-sapply(base5[,listnumeric],sd)

# Estandarizo solo las continuas y uno con las categoricas
base6<-scale(base5[,listnumeric], center = means, scale = sds)
numerocont<-which(colnames(base5)%in%listnumeric)
base7<-cbind(base6,base5[,-numerocont])
base7[1:5,1:4]
```

<div id='id6' />

#### **Selección de variables**

Se van a realizar 3 validaciones para la selección de variables, las cuales nos permiten identificar cuáles son las más predictivas. 

**Stepwise**: Stepwise va a ser la primera validación que vamos a realizar mediante una regresión y sus significancia estadística para agregar o eliminar posibles variables en el modelo final.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
full<-glm(factor(Attrition_Flag)~.,data=base7,family = binomial(link="logit"))
null<-glm(factor(Attrition_Flag)~1,data=base7,family = binomial(link="logit"))
```

| Selección de variables de Stepwise (Top 12) |
|------- |------------- |--------- |
| Dependent_count | Months_Inactive_12_mon | 	Total_Relationship_Count |
| Contacts_Count_12_mon | Gender.F | Total_Trans_Amt |
| Total_Revolving_Bal | Total_Trans_Ct | Total_Ct_Chng_Q4_Q1 |
| Credit_Limit  | Marital_Status.Married  |  |

**Step Repetida**: 

Esta función que permitede ste 20 nodos y un decay de 0.1

```{r, echo=FALSE, message=FALSE, warning=FALSE}
steprepetidobinaria<- function(data=data,vardep="vardep",
  listconti="listconti",
 sinicio=12345,sfinal=12359,porcen=0.8,criterio="BIC")
 {
library(MASS)
library(dplyr)
resultados<-data.frame(c())
data<-data[,c(listconti,vardep)]
formu1<-formula(paste("factor(",vardep,")~.",sep=""))
formu2<-formula(paste("factor(",vardep,")~1",sep=""))
listamodelos<-list()
for (semilla in sinicio:sfinal)
{
set.seed(semilla)
sample <- sample.int(n = nrow(data),
 size = floor(porcen*nrow(data)), replace = F)
train <- data[sample, ]
test  <- data[-sample, ]
full<-glm(formu1,data=train,family = binomial(link="logit"))
null<-glm(formu2,data=train,family = binomial(link="logit"))
if  (criterio=='AIC')
  {
  selec1<-stepAIC(null,scope=list(upper=full),
   direction="both",family = binomial(link="logit"),trace=FALSE)
  } 
else   if  (criterio=='BIC')
  {
 k1=log(nrow(train))
 selec1<-stepAIC(null,scope=list(upper=full),
  direction="both",family = binomial(link="logit"),k=k1,trace=FALSE)
  }
vec<-(names(selec1[[1]]))
if (length(vec)!=1)
{
# CAMBIOS
cosa<-as.data.frame(table(vec))
cosa<-as.data.frame(t(cosa))
colnames(cosa)<-vec
# 1) creo un vector con todas las variables input y ceros
# 2) voy aÃ±adiendo
cosa<-cosa[2,,drop=FALSE]
cosa<-cosa[,-(1),drop=FALSE]
cosa<- data.frame(lapply(cosa, function(x) as.numeric(as.character(x))))
cosa$id<-semilla
}
if (length(vec)==1)
{
cosa<-data.frame()
cosa[1,"id"]<-semilla
cosa$id<-as.integer(cosa$id)
}
vectormodelo<-list(names(cosa),semilla)
listamodelos<-append(listamodelos,vectormodelo)  
if (semilla==sinicio)
{
listamod<-cosa
}

else if (semilla!=sinicio)
{
 listamod<-suppressMessages(full_join(cosa,listamod,by = NULL, copy =TRUE))
}
}
listamod[is.na(listamod)] <- 0
nom<-names(listamod)
listamod$modelo<-""
for (i in 1:nrow(listamod))
{
 listamod[i,c("modelo")]<-""
 listamod[i,c("contador")]=0

  for (vari in nom)
  { 
   if (listamod[i,vari]==1)
   {
   listamod[i,c("modelo")]<-paste(listamod[i,c("modelo")],vari,collapse="",sep="+")
   listamod[i,c("contador")]=listamod[i,c("contador")]+1
   }
  
   }

}
 
listamod$modelo<-substring(listamod$modelo, 2)
tablamod<-as.data.frame(table(listamod$modelo))
names(tablamod)<-c("modelo","Freq")
tablamod<-tablamod[order(-tablamod$Freq,tablamod$modelo),]
nuevo<-listamod[,c("modelo","id","contador")]
uni<-full_join(tablamod,nuevo,by ="modelo", copy =TRUE)
uni= uni[!duplicated(uni$modelo),]
uni$semilla<-semilla
li1<-list()
# str(listamodelos)
for (i in 1:nrow(uni))
{
 for (j in 1:length(listamodelos))
 {
    if (uni[i,c("id")]==listamodelos[j][[1]])
  {
   k<-as.vector(listamodelos[j-1][[1]])
   length(k)<-length(k)-1
   li1<-c(li1,list(k))
   j=length(listamodelos)
  }
 } 

}

 uni$semilla<-NULL
 uni$id<-NULL
 return(list(uni,li1))

}
```


```{r, echo=FALSE, message=FALSE, warning=FALSE}
data<-base7
listconti<-c("Customer_Age", "Dependent_count", "Months_on_book", "Total_Relationship_Count",
"Months_Inactive_12_mon", "Contacts_Count_12_mon", "Credit_Limit", 
"Total_Revolving_Bal", "Avg_Open_To_Buy", "Total_Amt_Chng_Q4_Q1", 
"Total_Trans_Amt", "Total_Trans_Ct", "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio", 
"Gender.F", "Gender.M", "Education_Level.College", 
"Education_Level.Doctorate", "Education_Level.Graduate", "Education_Level.High School", 
"Education_Level.Post-Graduate", "Education_Level.Uneducated", 
"Marital_Status.Divorced", "Marital_Status.Married", "Marital_Status.Single", 
"Income_Category.$120K +", "Income_Category.$40K - $60K", "Income_Category.$60K - $80K", 
"Income_Category.$80K - $120K", "Income_Category.Less than $40K", 
"Card_Category.Blue", "Card_Category.Gold", "Card_Category.Platinum", 
"Card_Category.Silver")

lista<-steprepetidobinaria(data=data,
                           vardep=vardep,listconti=listconti,sinicio=12345,
                           sfinal=12395,porcen=0.8,criterio="BIC")
```

```{r, echo=FALSE, message=FALSE, warning=FALSE}
tabla<-lista[[1]]
dput(lista[[2]][[1]])
dput(lista[[2]][[2]])

# Vamos a quedarnos con estos modelos
mod1<-c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", 
"Dependent_count")

control<-trainControl(method = "repeatedcv",number=4,repeats = 5,
                      classProbs=TRUE,savePredictions = "all") 

avnnetgrid <-expand.grid(size=c(5,10,15,20),
                         decay=c(0.01,0.1,0.001),bag=FALSE)

redavnnet<- train(Attrition_Flag~ Total_Trans_Ct+Total_Revolving_Bal+Total_Relationship_Count+ Total_Ct_Chng_Q4_Q1+ Total_Trans_Amt+ Months_Inactive_12_mon+Contacts_Count_12_mon+Gender.F+Marital_Status.Married+Dependent_count,
                  data=data,
                  method="avNNet",linout = FALSE,maxit=100,
                  trControl=control,tuneGrid=avnnetgrid,
                  repeats=5)
redavnnet
```

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(plyr)
detach(package:plyr)
library(dummies)
library(MASS)
library(reshape)
library(caret)
library(dplyr)
library(pROC)

# CRUZADA LOGISTICA

cruzadalogistica <- function(data=data,vardep=NULL,
 listconti=NULL,listclass=NULL,grupos=4,sinicio=1234,repe=5)
{
 
if (any(listclass==c(""))==FALSE)
    {
   for (i in 1:dim(array(listclass))) {
    numindi<-which(names(data)==listclass[[i]])
    data[,numindi]<-as.character(data[,numindi])
    data[,numindi]<-as.factor(data[,numindi])
   }
  }   
  
  data[,vardep]<-as.factor(data[,vardep])
  
  # Creo la formula para la logistica
  
  if (any(listclass==c(""))==FALSE)
  {
   koko<-c(listconti,listclass)
  }  else   {
   koko<-c(listconti)
  }
 
  modelo<-paste(koko,sep="",collapse="+")
  formu<-formula(paste(vardep,"~",modelo,sep=""))
  
  formu 
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  regresion <- train(formu,data=data,
   trControl=control,method="glm",family = binomial(link="logit"))                  
  preditest<-regresion$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  return(medias)
  
 }

# CRUZADA avNNet

cruzadaavnnetbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",grupos=4,sinicio=1234,repe=5,
  size=c(5),decay=c(0.01),repeticiones=5,itera=100,trace=TRUE)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste(vardep,"~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  avnnetgrid <-  expand.grid(size=size,decay=decay,bag=FALSE)
  
  avnnet<- train(formu,data=databis,
   method="avNNet",linout = FALSE,maxit=itera,repeats=repeticiones,
   trControl=control,tuneGrid=avnnetgrid,trace=trace)
  
  print(avnnet$results)
  
  preditest<-avnnet$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n

   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"
  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
  return(medias)
  
 }
```

```{r, message=FALSE, echo=FALSE, warning=FALSE}
medias1<-cruzadalogistica(data=data,
                          vardep="Attrition_Flag",
                          listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                          listclass=c(""), grupos=4,sinicio=1234,repe=5)
medias1$modelo="Logistica1"

medias2<-cruzadaavnnetbin(data=data,
                          vardep="Attrition_Flag",listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                          listclass=c(""),grupos=4,sinicio=1234,repe=5,
                          size=c(20),decay=c(0.1),repeticiones=5,itera=200)
medias2$modelo="avnnet1"

union1<-rbind(medias1,medias2)
par(cex.axis=0.8,mfrow=c(1,2))
boxplot(data=union1,tasa~modelo,main="TASA FALLOS",col="pink")
boxplot(data=union1,auc~modelo,main="AUC",col="pink")
```

| Selección de variables Step Repetido Binario |
|------- |------------- |--------- |
| Total_Trans_Ct | Total_Trans_Amt | 	Total_Revolving_Bal |
| Total_Ct_Chng_Q4_Q1 | Months_Inactive_12_mon | Contacts_Count_12_mon |
| Gender.F | Total_Relationship_Count | Marital_Status.Married |
| Dependent_count |  |  |


**Árbol de decisiones**: El split que se está usando para el árbol es "Gini" ya que entre mayor es el valor entonces mayor es su homogenidad, y explicando un poco el árbol, la variable que predomina es "Total Trans Ct", es decir, la cantidad de transacciones en el último año donde el 84% que sí tiene entonces valida el monto de transacciones el mismo periodo y si es inferior a -0.44 entonces el 16% se tiene que revisar el saldo giratorio de la tarjeta.

```{r, echo=FALSE, message=FALSE, warning=FALSE}
library(rpart.plot)
arbol1 <- rpart(factor(Attrition_Flag) ~., data = base7,
                minbucket =30,method = "class",parms=list(split="gini"))
rpart.plot(arbol1,extra=105,nn=TRUE,tweak=0.7)
```

| Selección de variables del Árbol de decisión |
|------- |------------- |
| Total_Trans_Ct | Total_Trans_Amt |
| Total_Ct_Chng_Q4_Q1 | Total_Revolving_Bal |
| Customer_Age | Total_Relationship_Count |



**Selección Variables Final**: 

De acuerdo con el análisis que se realizó con la selección de variables mediante los 3 métodos y en resumen se lograron identificar 10 variables predictivas en el Stepwise, StepRepetido y Árbol.

| Selección de variables Final |
|------- |------------- |--------- |
| Total_Trans_Ct | Total_Revolving_Bal | Gender.F	 |
| Total_Trans_Amt | Months_Inactive_12_mon | Contacts_Count_12_mon |
| Total_Ct_Chng_Q4_Q1 | Total_Relationship_Count | Marital_Status.Married |
| Dependent_count |  |  |

Es decir, las variables del cuadro anterior serán usadas en los modelos de Machine Learning para la predicción de los clientes que van a abandonar el banco y no continuar más con la tarjeta de crédito. 

<div id='id7' />

#### **Modelización**

**Regresión Logística**

Comenzamos la modelización con la Regresión logística y las 10 variables que seleccionamos en el paso anterior. 

```{r, warning=FALSE, echo=FALSE, message=FALSE}
registerDoParallel(makeCluster(7))
```

```{r, warning=FALSE, echo=FALSE, message=FALSE}
#LOGISTICA
logistica<- train(Attrition_Flag~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,data=data,
             method="glm",trControl=control)

#Matriz de confusión
sal<-logistica$pred
salconfu<-confusionMatrix(sal$pred,sal$obs)

#AUC
curvaroc<-roc(response=sal$obs,predictor=sal$Yes)
auc<-curvaroc$auc
```

El resultado del modelo es el siguiente: 

|Accuracy  | Sensitivity | Specificity | AUC |
|------- |------------- |--------- |--------- |
| 90.45% | 96.59% | 58.39% | 92.3% |

El resultado es positivo ya que los cuatro indicadores tienen un buen porcentaje, sin embargo, la Especificidad se puede mejorar para que el porcentaje de aciertos sea mayor.


**Redes Neuronales**

La red neuronal se está realizando con un parámetros de 5 a 20 nodos y un decay entre 0.001 a 0.1, incluso se establece un maxit de 100 y se define un linout "False" por ser una variable dependiente dinaria. 

```{r, warning=FALSE, echo=FALSE, message=FALSE}
avnnetgrid_modelo <-expand.grid(size=c(5,10,15,20),
                         decay=c(0.01,0.1,0.001),bag=FALSE)

redavnnet_modelo<- train(Attrition_Flag~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,
                  data=data,method="avNNet",linout = FALSE,maxit=100,
                  trControl=control,tuneGrid=avnnetgrid_modelo, repeats=5)

sal_red<-redavnnet_modelo$pred
sal_red_confu<-confusionMatrix(sal_red$pred,sal_red$obs)

curvaroc<-roc(response=sal_red$obs,predictor=sal_red$Yes)
auc<-curvaroc$auc
plot(roc(response=sal_red$obs,predictor=sal_red$Yes))

```


El resultado de Redes neuronales es el siguiente: 

|Accuracy  | Sensitivity | Specificity | AUC |
|------- |------------- |--------- |--------- |
| 94.4% | 98.28% | 74.1% | 97.2% |

Este modelo mejoró más sus indicadores en comparación a Regresión logística, sin embargo, debemos hacer un comparativo final con todos los modelos. De igual manera los parámetros que recomienda son 20 nodos y decay de 0.1.

**Gradient Boosting**

Otra técnica de aprendizaje es Gradient boosting, el mismo usa diferentes parámetros como Shrinkage, número de nodos, cantidad de árboles y depth, y en el ejercicio vamos a aplicar nuevamente la Validación Cruzada Repetitiva y un rango de parámetros para que el modelo recomiende los parámetros que mayor Accuracy y otros indicadores que debemos considerar para saber si el modelo es predictivo.

```{r, warning=FALSE, echo=FALSE, message=FALSE}
gbmgrid<-expand.grid(shrinkage=c(0.1,0.05,0.03,0.01,0.001),
                     n.minobsinnode=c(5,10,20),
                     n.trees=c(100,300,500),
                     interaction.depth=c(2))
gbm<- train(factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,data=data,
            method="gbm",trControl=control,tuneGrid=gbmgrid,
            distribution="bernoulli", bag.fraction=1,verbose=FALSE)
plot(gbm)
```

De acuerdo con el modelo anterior donde tenía varios parámetros, se genera el modelo con las recomendaciones del anterior, es decir, 500 árboles, 2 iteracciones, 10 nodos y 0.1 de shrinkage. Sin embargo, el nodo de 5 parece tener el mismo Accuracy entonces es preferible usar el que tenga menos nodos. 

Además, las variables más importantes son las relacionadas a las transacciones de la tarjeta de crédito como:

```{r, warning=FALSE, echo=FALSE, message=FALSE}
gbmgrid_validacion<-expand.grid(shrinkage=c(0.1),
                     n.minobsinnode=c(10),
                     n.trees=c(500),
                     interaction.depth=c(2))
gbm_validacion<- train(factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,data=data,
            method="gbm",trControl=control,tuneGrid=gbmgrid_validacion,
            distribution="bernoulli", bag.fraction=1,verbose=FALSE)

# IMPORTANCIA DE VARIABLES
par(cex=1.3)
par(cex=1.5,las=2, mfrow=c(2,2))
barplot(tabla$rel.inf,names.arg=row.names(tabla))

sal_gbm<-gbm_validacion$pred
sal_gbm_confu<-confusionMatrix(sal_gbm$pred,sal_gbm$obs)
```


```{r, warning=FALSE, echo=FALSE, message=FALSE}
library(plyr)
detach(package:plyr)
library(dummies)
library(MASS)
library(reshape)
library(caret)
library(dplyr)
library(pROC)


cruzadagbmbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  n.minobsinnode=20,shrinkage=0.1,n.trees=100,interaction.depth=2)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  
  
   gbmgrid <-expand.grid(n.minobsinnode=n.minobsinnode,
    shrinkage=shrinkage,n.trees=n.trees,
    interaction.depth=interaction.depth)
  
  gbm<- train(formu,data=databis,
   method="gbm",trControl=control,
   tuneGrid=gbmgrid,distribution="bernoulli",verbose=FALSE)
  
  print(gbm$results)
  
  preditest<-gbm$pred
  
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
  return(medias)
  
 }
```


```{r, warning=FALSE, echo=FALSE, message=FALSE}
medias_boosting1<-cruzadagbmbin(data=data, vardep="Attrition_Flag",
                                listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                                listclass=c(""),
                                grupos=4,sinicio=1234,repe=5,
                                n.minobsinnode=10,shrinkage=0.1,n.trees=200,interaction.depth=2)

medias_boosting1$modelo="gbm1"

medias_boosting2<-cruzadagbmbin(data=data, vardep="Attrition_Flag",
                                listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                                listclass=c(""),
                                grupos=4,sinicio=1234,repe=5,
                                n.minobsinnode=10,shrinkage=0.1,n.trees=300,interaction.depth=2)

medias_boosting2$modelo="gbm2"

medias_boosting3<-cruzadagbmbin(data=data, vardep="Attrition_Flag",
                                listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                                listclass=c(""),
                                grupos=4,sinicio=1234,repe=5,
                                n.minobsinnode=10,shrinkage=0.1,n.trees=400,interaction.depth=2)

medias_boosting3$modelo="gbm3"

medias_boosting4<-cruzadagbmbin(data=data, vardep="Attrition_Flag",
                                listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                                listclass=c(""),
                                grupos=4,sinicio=1234,repe=5,
                                n.minobsinnode=10,shrinkage=0.1,n.trees=500,interaction.depth=2)

medias_boosting4$modelo="gbm4"

union_boosting<-rbind(medias_boosting1,medias_boosting2,medias_boosting3,medias_boosting4)

par(cex.axis=0.8)
boxplot(data=union_boosting,tasa~modelo,main="TASA FALLOS",col="pink")
boxplot(data=union_boosting,auc~modelo,main="AUC",col="pink")

curvaroc<-roc(response=sal_gbm$obs,predictor=sal_gbm$Yes)
auc<-curvaroc$auc
plot(roc(response=sal_red$obs,predictor=sal_red$Yes))
```

En resumen G. Boosting ha dado un importante resultado predictivo ya que la precisión es de un 96%.

|Accuracy  | Sensitivity | Specificity | AUC |
|------- |------------- |--------- |--------- |
| 96.2% | 98.28% | 84.2% | 98.4% |


**Bagging**

El método de bagging es donde los algoritmos simples son usados en paralelo, es decir, el error se puede reducir al promediar las salidas del modelo. Además, Bagging es conocido también por ser familia de Random Forest, solamente que este usa en mtry el número de variables. 

En este modelo vamos a usar los parámetros del número de árboles que siempre se recomiendan entre 100 a 200 ya que después de eso es estable el resultado con un número mayor de árboles.

```{r, warning=FALSE, echo=FALSE, message=FALSE}
library(plyr)
detach(package:plyr)
library(dummies)
library(MASS)
library(reshape)
library(caret)
library(dplyr)
library(pROC)

cruzadagbmbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  n.minobsinnode=20,shrinkage=0.1,n.trees=100,interaction.depth=2)
 { 
  # PreparaciÃ³n del archivo
  # b)pasar las categÃ³ricas a dummies
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  # c)estandarizar las variables continuas
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  # Estandarizo solo las continuas y uno con las categoricas
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  databis[,vardep]<-as.factor(databis[,vardep])
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  # Preparo caret   
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
   gbmgrid <-expand.grid(n.minobsinnode=n.minobsinnode,
    shrinkage=shrinkage,n.trees=n.trees,
    interaction.depth=interaction.depth)
  
  gbm<- train(formu,data=databis,
   method="gbm",trControl=control,
   tuneGrid=gbmgrid,distribution="bernoulli",verbose=FALSE)
  print(gbm$results)
  preditest<-gbm$pred
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"
  # Unimos la info de auc y de tasafallos
  medias$auc<-mediasbis$auc
  return(medias)
 }


cruzadaarbolbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  cp=c(0),minbucket =20)
 { 
  # PreparaciÃ³n del archivo
  # b)pasar las categÃ³ricas a dummies
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  databis[,vardep]<-as.factor(databis[,vardep])
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  arbolgrid <-  expand.grid(cp=cp)
  arbol<- train(formu,data=databis,
   method="rpart",trControl=control,
   tuneGrid=arbolgrid,minbucket=minbucket)
  print(arbol$results)
  preditest<-arbol$pred
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"
  # Unimos la info de auc y de tasafallos
  medias$auc<-mediasbis$auc
  return(medias)
 }


cruzadarfbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,nodesize=20,
  mtry=2,ntree=50,replace=TRUE,sampsize=1)
 { 
  # if  (sampsize==1)
  # {
  #  sampsize=floor(nrow(data)/(grupos-1))
  # }
  # PreparaciÃ³n del archivo
  # b)pasar las categÃ³ricas a dummies
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  databis[,vardep]<-as.factor(databis[,vardep])
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  rfgrid <-expand.grid(mtry=mtry)

  if  (sampsize==1)
  {
    rf<- train(formu,data=databis,
   method="rf",trControl=control,
   tuneGrid=rfgrid,nodesize=nodesize,replace=replace,ntree=ntree)
  }
  
else  if  (sampsize!=1)
  {
    rf<- train(formu,data=databis,
   method="rf",trControl=control,
   tuneGrid=rfgrid,nodesize=nodesize,replace=replace,sampsize=sampsize,
   ntree=ntree)
 }
  print(rf$results)
  preditest<-rf$pred
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"
  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"
  # Unimos la info de auc y de tasafallos
  medias$auc<-mediasbis$auc
  return(medias)
 }

```

También se van a realizar diferentes variaciones en los parámetros para ver cómo se comporta el modelo como tal, y el resultado de las 5 pruebas es el Bagging número 4 ya que posee la menor tasa de fallos y posee un alto porcentaje de AUC, sin embargo, está por debajo del Accuracy de Boosting por un 1 punto porcentual.

Los parámetros de este cuarto prueba es: "rupos=4,sinicio=1234,repe=5,nodesize=10, mtry=10,ntree=200,replace=TRUE,sampsize=900", con respecto al último punto se usó el 8.8% del tamaño de la base.

```{r, warning=FALSE, echo=FALSE, message=FALSE}
library(randomForest)
rfgrid<-expand.grid(mtry=c(10))

set.seed(12345)

rf_bagging<- train(data=data,
           factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,
           method="rf",trControl=control,tuneGrid=rfgrid,
           linout = FALSE,ntree=200,nodesize=10,replace=TRUE)

# PARA PLOTEAR EL ERROR OOB A MEDIDA QUE AVANZAN LAS ITERACIONES
# SE USA DIRECTAMENTE EL PAQUETE randomForest

rfbis_bagging<-randomForest(factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,
                    data=data,
                    mtry=10,ntree=200,sampsize=500,nodesize=10,replace=TRUE)

sal_bagging<-rf_bagging$pred
sal_bagging_confu<-confusionMatrix(sal_bagging$pred,sal_bagging$obs)

#Medias bagging
medias_bagging1<-cruzadarfbin(data=data, vardep="Attrition_Flag",
                              listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
            listclass=c(""),grupos=4,sinicio=1234,repe=5,nodesize=10,
            mtry=10,ntree=100,replace=TRUE,sampsize=500)

medias_bagging1$modelo="bagging1"

medias_bagging2<-cruzadarfbin(data=data, vardep="Attrition_Flag",
                              listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                              listclass=c(""),grupos=4,sinicio=1234,repe=5,nodesize=10,
                              mtry=10,ntree=100,replace=TRUE,sampsize=700)

medias_bagging2$modelo="bagging2"

medias_bagging3<-cruzadarfbin(data=data, vardep="Attrition_Flag",
                              listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                              listclass=c(""),grupos=4,sinicio=1234,repe=5,nodesize=10,
                              mtry=10,ntree=200,replace=TRUE,sampsize=800)

medias_bagging3$modelo="bagging3"

medias_bagging4<-cruzadarfbin(data=data, vardep="Attrition_Flag",
                              listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                              listclass=c(""),grupos=4,sinicio=1234,repe=5,nodesize=10,
                              mtry=10,ntree=200,replace=TRUE,sampsize=900)

medias_bagging4$modelo="bagging4"

medias_bagging5<-cruzadarfbin(data=data, vardep="Attrition_Flag",
                              listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                              listclass=c(""),grupos=4,sinicio=1234,repe=5,nodesize=10,
                              mtry=10,ntree=100,replace=TRUE,sampsize=1000)

medias_bagging5$modelo="bagging5"

union_bagging<-rbind(medias_bagging1,medias_bagging2,medias_bagging3,medias_bagging4,medias_bagging5)

par(cex.axis=0.8)
boxplot(data=union_bagging,tasa~modelo,main="TASA FALLOS",col="pink")
boxplot(data=union_bagging,auc~modelo,main="AUC",col="pink")
```


**RANDOM FOREST**

Los bosques aleatorios es otro modelo conocido en Machine Learning por su predicitivdad, lo cual hace que el modelo tenga estabilidad y es similar a Bagging pero en este caso usa "Mtry" usa todo el número de variables en cada partición de cada árbol.

Se recomienda usar un número de árboles pequeño (100 a 200) pero en este caso vamos a probar el modelo con un árbol de 500 pero un número mayor hace que el modelo tarde más en ejecutarse por el algoritmo que aplica y va a tener casi que el mismo resultado. En el siguiente gráfico se va a poder visualizar el número de árboles que ya se estabilizan apartir del 300 a 500.

```{r, warning=FALSE, echo=FALSE, message=FALSE}
rfgrid_random<-expand.grid(mtry=c(3,4,5,6,7,8,9,10))

rf_random<- train(factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,data=data,
           method="rf",trControl=control,tuneGrid=rfgrid_random,
           linout = FALSE,ntree=500,nodesize=10,replace=TRUE,
           importance=TRUE)

#El modelo anterior recomienda mtry=4
rfbis_random<-randomForest(factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,
                    data=data,
                    mtry=4,ntree=500,nodesize=10,replace=TRUE)

plot(rfbis_random$err.rate[,1])

sal_random<-rf_random$pred
sal_random_confu<-confusionMatrix(sal_random$pred,sal_random$obs)
```

En Random Forest también se va a realizar el estudio de medias, es decir, se van a probar varios modelos para que compitan entre sí teniendo diferentes parámetros, y así conocer el mejor para el modelo.

El resultado del mejor Random es de 98% de AUC y una tasa de error del 4.9%, teniendo en cuenta estos parámetros: Wgrupos=4,sinicio=1234,repe=5,nodesize=10, mtry=4,ntree=500,replace=TRUE,sampsize=1000".  

```{r, warning=FALSE, echo=FALSE, message=FALSE}
#Medias Random Forest

medias_random1 <- cruzadarfbin(data=data, 
                      vardep="Attrition_Flag",listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                      listclass=c(""),
                      grupos=4,sinicio=1234,repe=5,nodesize=10,
                      mtry=4,ntree=200,replace=TRUE,sampsize=400)

medias_random1$modelo="Random1"

medias_random2 <- cruzadarfbin(data=data, 
                               vardep="Attrition_Flag",listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                               listclass=c(""),
                               grupos=4,sinicio=1234,repe=5,nodesize=10,
                               mtry=4,ntree=300,replace=TRUE,sampsize=600)

medias_random2$modelo="Random2"

medias_random3 <- cruzadarfbin(data=data, 
                               vardep="Attrition_Flag",listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                               listclass=c(""),
                               grupos=4,sinicio=1234,repe=5,nodesize=10,
                               mtry=4,ntree=400,replace=TRUE,sampsize=800)

medias_random3$modelo="Random3"


medias_random4 <- cruzadarfbin(data=data, 
                               vardep="Attrition_Flag",listconti=c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count"),
                               listclass=c(""),
                               grupos=4,sinicio=1234,repe=5,nodesize=10,
                               mtry=4,ntree=500,replace=TRUE,sampsize=1000)

medias_random4$modelo="Random4"
medias_random4

curvaroc<-roc(response=sal_random$obs,predictor=sal_random$Yes)
auc<-curvaroc$auc

union_random<-rbind(medias_random1,medias_random2,medias_random3,medias_random4)

par(cex.axis=0.8, mfrow=c(1,2))
boxplot(data=union_random,tasa~modelo,main="TASA FALLOS",col="pink")
boxplot(data=union_random,auc~modelo,main="AUC",col="pink")

```


**SVM**

Este algoritmo es un aprendizaje supervisado y existen varios métodos pero en este estudio vamos a realizar el Lineal y el Radial, y no se realizó el Polynomial porque el resultado de la precisión del modelo fue baja.

```{r, warning=FALSE, echo=FALSE, message=FALSE}
SVMgrid<-expand.grid(C=c(0.01,0.05,0.1,0.2,0.5,1,2,5,10))

SVM<- train(data=data,factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,
            method="svmLinear",trControl=control,
            tuneGrid=SVMgrid,verbose=FALSE)
SVM$results

sal_svm<-SVM$pred
sal_svm_confu<-confusionMatrix(sal_svm$pred,sal_svm$obs)
sal_svm_confu


curvaroc<-roc(response=sal_svm$obs,predictor=sal_svm$Yes)
auc<-curvaroc$auc
auc
plot(roc(response=sal_svm$obs,predictor=sal_svm$Yes))

```

|Accuracy  | Sensitivity | Specificity | AUC |
|------- |------------- |--------- |--------- |
| 90.54% | 96.71% | 58.32% | 91.8% |


```{r, warning=FALSE, echo=FALSE, message=FALSE}
#  SVM RBF
SVMgrid<-expand.grid(C=c(0.5,1,2,5),
                     sigma=c(0.0001,0.005,0.01,0.05))

SVM_radial<- train(data=data,factor(Attrition_Flag)~Total_Trans_Ct+Total_Revolving_Bal+Gender.F+Total_Trans_Amt+Months_Inactive_12_mon+Contacts_Count_12_mon+Total_Ct_Chng_Q4_Q1+Total_Relationship_Count+Marital_Status.Married+Dependent_count,
            method="svmRadial",trControl=control,
            tuneGrid=SVMgrid,verbose=FALSE)

SVM_radial$results

sal_svm_radial<-SVM_radial$pred
sal_svmradial_confu<-confusionMatrix(sal_svm_radial$pred,sal_svm_radial$obs)
sal_svmradial_confu


curvaroc<-roc(response=sal_svm_radial$obs,predictor=sal_svm_radial$Yes)
auc<-curvaroc$auc
auc
plot(roc(response=sal_svm_radial$obs,predictor=sal_svm_radial$Yes))

dat_radial<-as.data.frame(SVM_radial$results)

ggplot(dat_radial, aes(x=factor(C), y=Accuracy, 
                color=factor(sigma)))+ 
  geom_point(position=position_dodge(width=0.5),size=3)
```
|Accuracy  | Sensitivity | Specificity | AUC |
|------- |------------- |--------- |--------- |
| 91.5% | 96.3% | 66.76% | 93.5% |

Los dos modelos de SVM no tuvieron un gran desempeño en el resultado, ya que fueron los modelos que menor porcentaje de Accuracy ya que se vio afectado por la Especificidad que no sobrepasaron el 70%.

<div id='id8' />

**Ensamblado**

El ensamble de modelos ayuda al rendimiento de los modelos de Machine Learning, principalmente para mejorar su precisión, mediante una combinación de predicciones de varios modelos donde promedio el error y produce una mejor predicción.

```{r, warning=FALSE, echo=FALSE, message=FALSE}
library(corrplot)
library(ggplot2)

archivo<-data

vardep<-"Attrition_Flag"
listconti<-c("Total_Trans_Ct", "Total_Revolving_Bal", "Total_Relationship_Count", "Total_Ct_Chng_Q4_Q1", "Total_Trans_Amt", "Months_Inactive_12_mon", "Contacts_Count_12_mon", "Gender.F", "Marital_Status.Married", "Dependent_count")
listclass<-c("")
grupos<-4
sinicio<-1234
repe<-5


# APLICACION CRUZADAS PARA ENSAMBLAR
medias1<-cruzadalogistica(data=archivo,
                          vardep=vardep,listconti=listconti,
                          listclass=listclass,grupos=grupos,sinicio=sinicio,repe=repe)

medias1bis<-as.data.frame(medias1[1])
medias1bis$modelo<-"Logistica"
predi1<-as.data.frame(medias1[[2]])
predi1$logi<-predi1$Yes
```

```{r, warning=FALSE, echo=FALSE, message=FALSE}
library(plyr)
detach(package:plyr)
library(dummies)
library(MASS)
library(reshape)
library(caret)
library(dplyr)
library(pROC)


# *********************************
# CRUZADA LOGISTICA
# ********************************* 

cruzadalogistica <- function(data=data,vardep=NULL,
 listconti=NULL,listclass=NULL,grupos=4,sinicio=1234,repe=5)
{

  if (any(listclass==c(""))==FALSE)
  {
   for (i in 1:dim(array(listclass))) {
    numindi<-which(names(data)==listclass[[i]])
    data[,numindi]<-as.character(data[,numindi])
    data[,numindi]<-as.factor(data[,numindi])
   }
  }   
  
  data[,vardep]<-as.factor(data[,vardep])
  
  # Creo la formula para la logistica
  
if (any(listclass==c(""))==FALSE)
  {
   koko<-c(listconti,listclass)
  }  else   {
   koko<-c(listconti)
  }
 
  modelo<-paste(koko,sep="",collapse="+")
  formu<-formula(paste(vardep,"~",modelo,sep=""))
  
  formu 
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  regresion <- train(formu,data=data,
   trControl=control,method="glm",family = binomial(link="logit"))                  
  preditest<-regresion$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
 return(list(medias,preditest))
  
 }




# *********************************
# CRUZADA avNNet
# **************

cruzadaavnnetbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",grupos=4,sinicio=1234,repe=5,
  size=c(5),decay=c(0.01),repeticiones=5,itera=100,trace=FALSE)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste(vardep,"~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  avnnetgrid <-  expand.grid(size=size,decay=decay,bag=FALSE)
  
  avnnet<- train(formu,data=databis,
   method="avNNet",linout = FALSE,maxit=itera,repeats=repeticiones,
   trControl=control,tuneGrid=avnnetgrid,trace=trace)
  
  print(avnnet$results)
  
  preditest<-avnnet$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
   return(list(medias,preditest))
  
 }


# *********************************
# CRUZADA Random Forest
# ******************************


cruzadarfbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,nodesize=20,
  mtry=2,ntree=50,replace=TRUE,sampsize=1)
 { 
  
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  rfgrid <-expand.grid(mtry=mtry)
  
      if  (sampsize==1)
  {
    rf<- train(formu,data=databis,
   method="rf",trControl=control,
   tuneGrid=rfgrid,nodesize=nodesize,replace=replace,ntree=ntree)
  }
  
else  if  (sampsize!=1)
  {
    rf<- train(formu,data=databis,
   method="rf",trControl=control,
   tuneGrid=rfgrid,nodesize=nodesize,replace=replace,sampsize=sampsize,
   ntree=ntree)
 }
  
  print(rf$results)
  
  preditest<-rf$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
   return(list(medias,preditest))
  
 }

# ***************************************************************
# gbm : parÃ¡metros

# 
#     Number of Boosting Iterations (n.trees, numeric)
#     Max Tree Depth (max.depth, numeric)
#     Shrinkage (shrinkage, numeric)
#     Min. Terminal Node Size (n.minobsinnode, numeric)
#    
# ***************************************************************



cruzadagbmbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  n.minobsinnode=20,shrinkage=0.1,n.trees=100,interaction.depth=2)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  
  
   gbmgrid <-expand.grid(n.minobsinnode=n.minobsinnode,
    shrinkage=shrinkage,n.trees=n.trees,
    interaction.depth=interaction.depth)
  
  gbm<- train(formu,data=databis,
   method="gbm",trControl=control,
   tuneGrid=gbmgrid,distribution="bernoulli",verbose=FALSE)
  
  print(gbm$results)
  
  preditest<-gbm$pred
  
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
   return(list(medias,preditest))
  
 }
 

# ***************************************************************
# xgboost: parÃ¡metros

   # nrounds (# Boosting Iterations)
    # max_depth (Max Tree Depth)
    # eta (Shrinkage)
    # gamma (Minimum Loss Reduction)
    # colsample_bytree (Subsample Ratio of Columns)
    # min_child_weight (Minimum Sum of Instance Weight)
    # subsample (Subsample Percentage)
#    
# PONER linout = FALSE
# ***************************************************************

cruzadaxgbmbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  min_child_weight=20,eta=0.1,nrounds=100,max_depth=2,
  gamma=0,colsample_bytree=1,subsample=1,alpha=0,lambda=0,lambda_bias=0)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  xgbmgrid <-expand.grid( min_child_weight=min_child_weight,
  eta=eta,nrounds=nrounds,max_depth=max_depth,
  gamma=gamma,colsample_bytree=colsample_bytree,subsample=subsample)
  
  xgbm<- train(formu,data=databis,
   method="xgbTree",trControl=control,
   tuneGrid=xgbmgrid,objective = "binary:logistic",verbose=FALSE,
   alpha=alpha,lambda=lambda,lambda_bias=lambda_bias)
  
  print(xgbm$results)
  
  preditest<-xgbm$pred
  
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
   return(list(medias,preditest))
  
 }
 


# ***************************************************************
# svmLinear: parÃ¡metros

# Cost (C, numeric)     

# PONER linout = FALSE
# ***************************************************************


cruzadaSVMbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  C=1,replace=TRUE)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  SVMgrid <-expand.grid(C=C)
  
  SVM<- train(formu,data=databis,
   method="svmLinear",trControl=control,
   tuneGrid=SVMgrid,replace=replace)
  
  print(SVM$results)
  
  preditest<-SVM$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
   return(list(medias,preditest))
  
 }





cruzadaSVMbinPoly<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  C=1,degree=2,scale=1)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  SVMgrid <-expand.grid(C=C,degree=degree,scale=scale)
  
  SVM<- train(formu,data=databis,
   method="svmPoly",trControl=control,
   tuneGrid=SVMgrid,replace=replace)
  
  print(SVM$results)
  
  preditest<-SVM$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
   return(list(medias,preditest))
  
 }



# ***************************************************************
# svmRadial: parÃ¡metros

    # Sigma (sigma, numeric)
    # Cost (C, numeric)

# PONER linout = FALSE
# ***************************************************************



cruzadaSVMbinRBF<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  C=1,sigma=1)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  SVMgrid <-expand.grid(C=C,sigma=sigma)
  
  SVM<- train(formu,data=databis,
   method="svmRadial",trControl=control,
   tuneGrid=SVMgrid,replace=replace)
  
  print(SVM$results)
  
  preditest<-SVM$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
   return(list(medias,preditest))
  
 }

```


```{r, warning=FALSE, echo=FALSE, message=FALSE}
medias2<-cruzadaavnnetbin(data=archivo,
                          vardep=vardep,listconti=listconti,
                          listclass=listclass,grupos=grupos,sinicio=sinicio,repe=repe,
                          size=c(5),decay=c(0.1),repeticiones=5,itera=100)

medias2bis<-as.data.frame(medias2[1])
medias2bis$modelo<-"avnnet"
predi2<-as.data.frame(medias2[2])
predi2$avnnet<-predi2$Yes
```


```{r, warning=FALSE, echo=FALSE, message=FALSE}
medias3<-cruzadarfbin(data=archivo,
                      vardep=vardep,listconti=listconti,
                      listclass=listclass,grupos=grupos,sinicio=sinicio,repe=repe,
                      mtry=4,ntree=500,nodesize=10,replace=TRUE)

medias3bis<-as.data.frame(medias3[1])
medias3bis$modelo<-"rf"
predi3<-as.data.frame(medias3[2])
predi3$rf<-predi3$Yes
```


```{r, warning=FALSE, echo=FALSE, message=FALSE}
medias4<-cruzadagbmbin(data=archivo,
                       vardep=vardep,listconti=listconti,
                       listclass=listclass,grupos=grupos,sinicio=sinicio,repe=repe,
                       n.minobsinnode=20,shrinkage=0.1,n.trees=500,interaction.depth=2)

medias4bis<-as.data.frame(medias4[1])
medias4bis$modelo<-"gbm"
predi4<-as.data.frame(medias4[2])
predi4$gbm<-predi4$Yes
```

```{r, warning=FALSE, echo=FALSE, message=FALSE}
medias5<-cruzadarfbin(data=archivo,
                        vardep=vardep,listconti=listconti,
                        listclass=listclass,grupos=grupos,sinicio=sinicio,repe=repe,
                      nodesize=10,mtry=10,ntree=200,replace=TRUE)

medias5bis<-as.data.frame(medias5[1])
medias5bis$modelo<-"bagging"
predi5<-as.data.frame(medias5[2])
predi5$bagging<-predi5$Yes
```


```{r, warning=FALSE, echo=FALSE, message=FALSE}
cruzadaSVMbin<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  C=1,replace=TRUE)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  SVMgrid <-expand.grid(C=C)
  
  SVM<- train(formu,data=databis,
   method="svmLinear",trControl=control,
   tuneGrid=SVMgrid,replace=replace)
  
  print(SVM$results)
  
  preditest<-SVM$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
  return(medias)
  
 }

```

```{r, warning=FALSE, echo=FALSE, message=FALSE}
medias6<-cruzadaSVMbin(data=archivo,
                       vardep=vardep,listconti=listconti,
                       listclass=listclass,grupos=grupos,
                       sinicio=sinicio,repe=repe,C=0.05)

medias6bis<-as.data.frame(medias6[1])
medias6bis$modelo<-"svmLinear"
predi6<-as.data.frame(medias6[2])
predi6$svmLinear<-"svmLinear"
```

```{r, warning=FALSE, echo=FALSE, message=FALSE}
cruzadaSVMbinRBF<-
 function(data=data,vardep="vardep",
  listconti="listconti",listclass="listclass",
  grupos=4,sinicio=1234,repe=5,
  C=1,sigma=1)
 { 
  
  # PreparaciÃ³n del archivo
  
  # b)pasar las categÃ³ricas a dummies
  
if (any(listclass==c(""))==FALSE)
  {
   databis<-data[,c(vardep,listconti,listclass)]
   databis<- dummy.data.frame(databis, listclass, sep = ".")
  }  else   {
   databis<-data[,c(vardep,listconti)]
  }
  
  # c)estandarizar las variables continuas
  
  # Calculo medias y dtipica de datos y estandarizo (solo las continuas)
  
  means <-apply(databis[,listconti],2,mean)
  sds<-sapply(databis[,listconti],sd)
  
  # Estandarizo solo las continuas y uno con las categoricas
  
  datacon<-scale(databis[,listconti], center = means, scale = sds)
  numerocont<-which(colnames(databis)%in%listconti)
  databis<-cbind(datacon,databis[,-numerocont,drop=FALSE ])
  
  databis[,vardep]<-as.factor(databis[,vardep])
  
  formu<-formula(paste("factor(",vardep,")~.",sep=""))
  
  # Preparo caret   
  
  set.seed(sinicio)
  control<-trainControl(method = "repeatedcv",number=grupos,repeats=repe,
   savePredictions = "all",classProbs=TRUE) 
  
  # Aplico caret y construyo modelo
  
  SVMgrid <-expand.grid(C=C,sigma=sigma)
  
  SVM<- train(formu,data=databis,
   method="svmRadial",trControl=control,
   tuneGrid=SVMgrid,replace=replace)
  
  print(SVM$results)
  
  preditest<-SVM$pred
  
  preditest$prueba<-strsplit(preditest$Resample,"[.]")
  preditest$Fold <- sapply(preditest$prueba, "[", 1)
  preditest$Rep <- sapply(preditest$prueba, "[", 2)
  preditest$prueba<-NULL
  
  tasafallos<-function(x,y) {
   confu<-confusionMatrix(x,y)
   tasa<-confu[[3]][1]
   return(tasa)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   tabla<-table(preditest$Rep)
listarep<-c(names(tabla))
medias<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
tasa=1-tasafallos(paso1$pred,paso1$obs)  
medias<-rbind(medias,tasa)
}
names(medias)<-"tasa"

  
  # CalculamoS AUC  por cada RepeticiÃ³n de cv 
  # Definimnos funciÃ³n
  
  auc<-function(x,y) {
   curvaroc<-roc(response=x,predictor=y)
   auc<-curvaroc$auc
   return(auc)
  }
  
  # Aplicamos funciÃ³n sobre cada RepeticiÃ³n
  
  
   
   mediasbis<-data.frame()
for (repi in listarep) {
paso1<-preditest[which(preditest$Rep==repi),]
auc=auc(paso1$obs,paso1$Yes)
mediasbis<-rbind(mediasbis,auc)
}
names(mediasbis)<-"auc"

  
  # Unimos la info de auc y de tasafallos
  
  medias$auc<-mediasbis$auc
  
  return(medias)
  
 }
```


```{r,warning=FALSE, echo=FALSE, message=FALSE}
medias7<-cruzadaSVMbinRBF(data=archivo,
                          vardep=vardep,listconti=listconti,
                          listclass=listclass,grupos=grupos,
                          sinicio=sinicio,repe=repe,
                          C=10,sigma=0.05)

medias7bis<-as.data.frame(medias7[1])
medias7bis$modelo<-"svmRadial"
predi7<-as.data.frame(medias7bis[2])
predi7$svmRadial<-predi7$Yes
```

```{r,warning=FALSE, echo=FALSE, message=FALSE}
union_todos<-rbind(medias1bis,medias2bis,
              medias3bis,medias4bis,medias5bis,medias6bis,medias7bis)


par(cex.axis=0.8)
boxplot(data=union_todos,tasa~modelo,col="pink",main='TASA FALLOS')
boxplot(data=union_todos,auc~modelo,col="pink",main='AUC')

# CONSTRUCCION DE TODOS LOS ENSAMBLADOS
# SE UTILIZARON LOS ARCHIVOS SURGIDOS DE LAS FUNCIONES LLAMADOS predi1,...

unipredi<-cbind(predi1,predi2,predi3,predi4,predi5,predi6,predi7)

# Esto es para eliminar columnas duplicadas
unipredi<- unipredi[, !duplicated(colnames(unipredi))]

# Construccion de ensamblados, cambiar al gusto

unipredi$predi9<-(unipredi$logi+unipredi$avnnet)/2
unipredi$predi10<-(unipredi$logi+unipredi$rf)/2
unipredi$predi11<-(unipredi$logi+unipredi$gbm)/2
unipredi$predi12<-(unipredi$logi+unipredi$bagging)/2
unipredi$predi16<-(unipredi$avnnet+unipredi$rf)/2
unipredi$predi17<-(unipredi$avnnet+unipredi$gbm)/2
unipredi$predi18<-(unipredi$avnnet+unipredi$bagging)/2
unipredi$predi22<-(unipredi$rf+unipredi$gbm)/2
unipredi$predi23<-(unipredi$rf+unipredi$bagging)/2
unipredi$predi27<-(unipredi$gbm+unipredi$bagging)/2
unipredi$predi31<-(unipredi$logi+unipredi$avnnet+unipredi$rf)/3
unipredi$predi32<-(unipredi$logi+unipredi$avnnet+unipredi$gbm)/3
unipredi$predi33<-(unipredi$logi+unipredi$avnnet+unipredi$bagging)/3
unipredi$predi37<-(unipredi$logi+unipredi$rf+unipredi$gbm)/3
unipredi$predi38<-(unipredi$logi+unipredi$rf+unipredi$bagging)/3
unipredi$predi42<-(unipredi$logi+unipredi$gbm+unipredi$bagging)/3
unipredi$predi51<-(unipredi$rf+unipredi$gbm+unipredi$logi)/3
unipredi$predi54<-(unipredi$rf+unipredi$bagging+unipredi$gbm)/3
unipredi$predi56<-(unipredi$rf+unipredi$avnnet+unipredi$gbm)/3
unipredi$predi57<-(unipredi$rf+unipredi$avnnet+unipredi$bagging)/3
unipredi$predi59<-(unipredi$rf+unipredi$avnnet+unipredi$gbm)/3
unipredi$predi62<-(unipredi$avnnet+unipredi$gbm+unipredi$logi)/3
unipredi$predi70<-(unipredi$logi+unipredi$gbm+unipredi$avnnet)/3
unipredi$predi71<-(unipredi$logi+unipredi$rf+unipredi$avnnet)/3
unipredi$predi64<-(unipredi$logi+unipredi$rf+unipredi$gbm+unipredi$avnnet)/4
unipredi$predi65<-(unipredi$logi+unipredi$rf+unipredi$bagging+unipredi$avnnet)/4
unipredi$predi66<-(unipredi$logi+unipredi$rf+unipredi$bagging+unipredi$avnnet)/4
unipredi$predi68<-(unipredi$logi+unipredi$rf+unipredi$bagging+unipredi$avnnet+unipredi$gbm)/5

# Listado de modelos a considerar, cambiar al gusto

dput(names(unipredi))

listado<-c("logi","avnnet","rf","gbm","bagging","predi9", "predi10", 
"predi11", "predi12", "predi17", "predi16", "predi18", "predi22", 
"predi23", "predi27", "predi31", "predi32", "predi33", "predi37", 
"predi38", "predi42", "predi51", "predi54", "predi56", "predi57", 
"predi59", "predi62", "predi70", "predi71", "predi64", "predi65", 
"predi66", "predi68")

# Cambio a Yes, No, todas las predicciones

# Defino funcion tasafallos

tasafallos<-function(x,y) {
  confu<-confusionMatrix(x,y)
  tasa<-confu[[3]][1]
  return(tasa)
}

auc<-function(x,y) {
  curvaroc<-roc(response=x,predictor=y)
  auc<-curvaroc$auc
  return(auc)
}

# Se obtiene el numero de repeticiones CV y se calculan las medias por repe en
# el data frame medias0

repeticiones<-nlevels(factor(unipredi$Rep))
unipredi$Rep<-as.factor(unipredi$Rep)
unipredi$Rep<-as.numeric(unipredi$Rep)


medias0<-data.frame(c())
for (prediccion in listado)
{
  unipredi$proba<-unipredi[,prediccion]
  unipredi[,prediccion]<-ifelse(unipredi[,prediccion]>0.5,"Yes","No")
  for (repe in 1:repeticiones)
  {
    paso <- unipredi[(unipredi$Rep==repe),]
    pre<-factor(paso[,prediccion])
    archi<-paso[,c("proba","obs")]
    archi<-archi[order(archi$proba),]
    obs<-paso[,c("obs")]
    tasa=1-tasafallos(pre,obs)
    t<-as.data.frame(tasa)
    t$modelo<-prediccion
    auc<-auc(archi$obs,archi$proba)
    t$auc<-auc
    medias0<-rbind(medias0,t)
  }
}


# Finalmente boxplot

par(cex.axis=0.5,las=2)
boxplot(data=medias0,tasa~modelo,col="pink",main="TASA FALLOS")

# Para AUC se utiliza la variable auc del archivo medias0

boxplot(data=medias0,auc~modelo,col="pink",main="AUC")

# PRESENTACION TABLA MEDIAS

tablamedias<-medias0 %>%
  group_by(modelo) %>%
  summarize(tasa=mean(tasa))     

tablamedias<-tablamedias[order(tablamedias$tasa),]
tablamedias

# ORDEN DEL FACTOR MODELO POR LAS MEDIAS EN TASA
# PARA EL GRAFICO

medias0$modelo <- with(medias0,
                       reorder(modelo,tasa, mean))
par(cex.axis=0.7,las=2)
boxplot(data=medias0,tasa~modelo,col="pink", main='TASA FALLOS')

# ************************************
# PARA AUC
# ************************************

# PRESENTACION TABLA MEDIAS

tablamedias2<-medias0 %>%
  group_by(modelo) %>%
  summarize(auc=mean(auc))     

tablamedias2<-tablamedias2[order(-tablamedias2$auc),]
tablamedias2

# ORDENACIÃN DEL FACTOR MODELO POR LAS MEDIAS EN AUC
# PARA EL GRAFICO

medias0$modelo <- with(medias0,
                       reorder(modelo,auc, mean))
par(cex.axis=0.7,las=2)
boxplot(data=medias0,auc~modelo,col="pink", main='AUC')

```

Después de validar el ensamblado, la predicción que más se rescata es "Predi22", la cual es una combinación entre el modelo de Random Forest y Gradient Boosting, es decir, los dos mejores modelos que hemos generado ya que ambos tenían una precisión por encima del 95%.

En conclusión, podemos definir lo siguiente:

- Las características del cliente que más predominan en la selección de variables y modelo son las del comportamiento que tiene el cliente con la tarjeta, es decir, su uso en transacciones, monto, periodo de no uso, entre otros, ya que no se reportaron variables sociodemográficas o socioeconómicas significativas en el modelo predictivo.

- El ensamblado ayudó a generar una mejor precisión en el modelo con un 98%, lo cual es un gran resultado para predecir quién es el cliente que puede abandonar el banco y así proactivamente mejorar los servicios o aplicar algunas estrategia para que cambie de opinión.

- Los modelos más predictivos fueron Gradient Boosting, Random Forest y Bagging ya que estuvieron por encima del 95% de Accuracy, por eso el resultado tan positivo entre en el ensamblado por la combinación de dos grandes modelos.


```{r setup, include=FALSE}
knitr::opts_chunk$set(echo = TRUE)
```